<a href="https://colab.research.google.com/github/DavinciB/child_grooming_detector/blob/main/Data%20Prefiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import xml.etree.ElementTree as ET #for parsing the xml file to a tree
import datetime
#store the path to the folder containing the training data
train_data_path = r"/content/drive/MyDrive/online-grooming-detector-master/data/pan12-sexual-predator-identification-training-corpus-2012-05-01"
#parse the training data xml file
training_xml = ET.parse(train_data_path + '/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml')
#obtain the root of the tree we created above
root = training_xml.getroot()

In [ ]:
#Adding index attribute to conversations and changing line attributes of messages for ease of pre processing
i = 0
for conversation in root:
  conversation.set('index',i)   #adding an index value to each conversation in the tree
  i += 1
for conversation in root:
  i = 0
  for message in conversation:
    message.set('line',i) #changing the line value of each messsage in all conversation to be in ordered form
    i += 1

In [ ]:
#Before removing single author conversations
print(root[1][0][2].text)

In [ ]:
#single author 
conv_2_remove = [] #list to store the conversation id of conversations to be removed
authors = [] #list of unique authors in each conversations
init_len = len(root) #initial no of conversations

for conversation in root:
    authors.clear() #clear the authors list before going through each conversation
    for message in conversation: #taking individual messages in each conversation
        author = message.find('author').text #finding the author of the message
        if author not in authors: #if the author is not present in the list
            authors.append(author) #append if not

    if (len(authors)) <= 1 and conversation.get('id') not in conv_2_remove: #if no of authers less than 1 and conversatio id already not present in conv_2_remove
        conv_2_remove.append(conversation.get('id')) #append to the list if not

print(str(len(conv_2_remove))+" out of "+str(init_len)+" conversations contain single user")
sing_user = len(conv_2_remove) #no of single user conversations

In [ ]:
#Before removing low message count conversations
print(root[2][0][2].text)

In [ ]:
#Each user having 5 or less messages
for conversation in root: #taaking the conversation on at a time
    if conversation.get('id') in conv_2_remove: #check if id of conversation already present in conv_2_remove
        continue #skipping if true
    authors = {} #python dictionary of authors
    for message in conversation: #checking the author of each message in a conversation
        author = message.find('author').text
        if author in authors: #if present inrement the value
            authors[author] = authors[author] + 1
        else: #new author store the initial value as 1
            authors[author] = 1
    remove = True #initialize remove flag as true
    for author in authors:
        if authors[author] > 5:# checking if any auther have more than 5 messages if no single user have morethan 5 messages the conversation is removed
            remove = False #set flag faslse
    if remove is True and conversation.get('id') not in conv_2_remove: #check if flag true and conversation id not in list
        conv_2_remove.append(conversation.get('id')) #apppend if true
low_msg_count = len(conv_2_remove) - sing_user
print(str(low_msg_count)+" out of "+str(init_len)+" conversations have low message count")

In [ ]:
for conversation in root.findall('conversation'): #finds all the children of root having conversation tag ie. the conversations
    if conversation.get('id') in conv_2_remove: #if conversation id in list to be removed
        root.remove(conversation) #remove the child
print("The new root has a length of " + str(len(root)))
print(str(init_len - len(root))+" conversations removed")

In [ ]:
#resetting index attribute to conversations and changing line attributes of messages
i = 0
for conversation in root:
  conversation.set('index',i)   
  i += 1
for conversation in root:
  i = 0
  for message in conversation:
    message.set('line',i)
    i += 1

In [ ]:
#After removing single author conversations
print(root[1][0][2].text)

In [ ]:
#Afetr removing low message count conversations
print(root[2][0][2].text)

In [ ]:
#Removing sequences of meaningless characters
import re #regular expression operators for string matching
messages = [] #list of messages
count = 0
for conversation in root: #take eeach conversation
  messages.clear() #clear the message list before each iteration
  conv = conversation.get('id') #take the conversatio id 
  index = conversation.get('index') #take the conversation index for indexing
  for message in conversation: #consider each message
    text = message.find("text").text #take the text conyent of the message
    if text is None or len(text) < 20: #if no text present or length less than 20 skip
      continue
    match_str = re.findall("[\W_]", text) #pattern matching expression to find the string of non-word charecters
    if len(match_str) / len(text) > 0.6: #if the above string occupies more than 60% of the message it needs to be removed
      line = message.get('line') # take the line index from message
      messages.append(int(line)) #add the line index to the list
    for i in reversed(messages): #iterating from end to top
      try: #try statement for unexpected errors due to dataset structure
        root[int(index)].remove(root[int(index)][i]) #removing the messages
        count += 1
      except:
        continue #skipping the errors
print(str(count)+" messages has been removed")

In [ ]:
#Removing index and updating line attributes to str else it gives errror
#during converting to xml
for i in range(len(root)):
  root[i].attrib.pop('index') 
for conversation in root:
  i = 1 #in actual dataset line values started from 1 we chasnged it to start from 0 earlier for easier convenience
  for message in conversation:
    message.set('line',str(i))
    i += 1

In [ ]:
from xml.etree.ElementTree import ElementTree,tostring
tree = ElementTree(root)
tree.write(open(r'/content/drive/MyDrive/online-grooming-detector-master/data/svm_training_data/training_data.xml', 'wb')) #writing the preprocessed data back
print("Filtered data written!")